In [1]:
""" Writing my first Graduate Model test case.
Author : QiaoTian
Date : 16th Sep 2016
Revised: 16th Sep 2014

"""

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from minepy import MINE
from scipy.stats import pearsonr
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import cross_val_score

feature_cols = []
with open('genotype.csv') as f:
    firstline = f.readline()
    feature_cols = np.array(firstline.split(','))

genotype_df         = pd.read_csv('genotype.csv', header=0, names=feature_cols)
pheno_df            = pd.read_csv('phenotype.txt', header=None)
multi_phenos_df     = pd.read_csv('multi_phenos.txt', header=None)

# train data
train_df = genotype_df
label_df = pheno_df



K = 80 # feature number
train = train_df.as_matrix(feature_cols)
label = label_df.as_matrix().ravel()
train = SelectKBest(lambda X, Y: np.array(map(lambda x:pearsonr(x, Y), X.T)).T, k=K).fit_transform(train, label)

X_train, X_test, y_train, y_test = train_test_split(train, label, random_state=0, test_size=0.2)

# 相关系数

In [4]:
""" random foreset """
repeat = range(50)
accuracy_in_sample = []
accuracy_out_sample = []
#for n_est in x_vals:
for iter in repeat: # run 50 times
    #rf = RandomForestClassifier(n_estimators=n_est, max_depth = 5) # initialize
    rf = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                                max_depth=5, max_features='auto', max_leaf_nodes=None,
                                min_samples_leaf=10, min_samples_split=2,
                                min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
                                oob_score=False, random_state=None, verbose=0,
                                warm_start=False)
    rf.fit(X_train, y_train.ravel())        # fit the data to algorithm
    #y_train_pred = rf.predict(X_train_new)      # apply model to train data to get the error in sample
    #y_test_pred = rf.predict(X_test_new)        # apply model to test data to get the error out of sample
    #error_in.append(metrics.mean_absolute_error(y_train_pred, y_train))
    #error_out.append(metrics.mean_absolute_error(y_test_pred, y_test))
    accuracy_in_sample.append(rf.score(X_train, y_train))
    accuracy_out_sample.append(rf.score(X_test, y_test))

accuracy_in_sample  = np.array(accuracy_in_sample)
accuracy_out_sample = np.array(accuracy_out_sample)

print np.min(accuracy_in_sample)
print np.argmin(accuracy_in_sample)
print np.min(accuracy_out_sample)
print np.argmin(accuracy_out_sample)


import matplotlib.pyplot as plt
plot1 = plt.plot(repeat, accuracy_in_sample, color='green', label='Accuracy of in-sample')
plot2 = plt.plot(repeat, accuracy_out_sample, color='red', label='Accuracy of out-sample')
plt.xlabel('n_estimators = 200, number of features is ' + str(K))
plt.ylabel('Accuracy of training and testing')
plt.legend([plot1, plot2], ("training accuracy", "test accuracy"))# make legend
plt.show()

0.89375
39
0.615
4


/usr/local/anaconda2/lib/python2.7/site-packages/matplotlib/legend.py:633: UserWarning: Legend does not support [<matplotlib.lines.Line2D object at 0x118ab7c90>] instances.
A proxy artist may be used instead.
See: http://matplotlib.org/users/legend_guide.html#using-proxy-artist
  "#using-proxy-artist".format(orig_handle)
/usr/local/anaconda2/lib/python2.7/site-packages/matplotlib/legend.py:633: UserWarning: Legend does not support [<matplotlib.lines.Line2D object at 0x118ade550>] instances.
A proxy artist may be used instead.
See: http://matplotlib.org/users/legend_guide.html#using-proxy-artist
  "#using-proxy-artist".format(orig_handle)


# 交叉验证

In [27]:
K = 40
X = X.as_matrix(feature_cols)
y = y.as_matrix()
X = SelectKBest(lambda X, Y: np.array(map(lambda x:pearsonr(x, Y), X.T)).T, k=K).fit_transform(X, y)

""" random foreset """
error_in = []
error_out = []
n_est = 200
for it in range(200):
    rf = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                            max_depth=5, max_features='auto', max_leaf_nodes=None,
                            min_samples_leaf=10, min_samples_split=2,
                            min_weight_fraction_leaf=0.0, n_estimators=n_est, n_jobs=1,
                            oob_score=False, random_state=None, verbose=0,
                            warm_start=False)
    correctness = cross_val_score(rf, X, y, cv=5)
    error_out.append(correctness.mean())
error_out = np.array(error_out)

import matplotlib.pyplot as plt
#plt.plot(range(200), error_in, color='green', label='Error in Sample')
plt.plot(range(200), error_out, color='red', label='Error out of Sample')
plt.xlabel('n_estimators = 200, number of features is ' + str(K))
plt.ylabel('Error')
plt.show()

/usr/local/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(1000, 40)
(1000, 1)


IndexError: too many indices for array